Import mmsdk package:

In [2]:
from constants import SDK_PATH, DATA_PATH, WORD_EMB_PATH, CACHE_PATH
import sys

if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)

import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError

# create folders for storing the data
if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

Preparation import to modify mmsdk dataset alignment method

In [3]:
from mmsdk.mmdatasdk import log, computational_sequence
import numpy
import time
import struct
from tqdm import tqdm_notebook as tqdm

Inheritate from mmsdk with new methods: 

In [4]:

def save_htk_format(features, feature_type, folder, video_number):
    '''
    This function works for function align_upsampling_and_save. It save feature vectors of one video id into one 
    htk format file. 
    feature: feature of one video
    All files are of USER type.
    number of samples being consistent with number of samples in csd file
    sample bytes for COVAREP: 4 * 74 = 296
    sample period: 10 000.0 us (100000)
    paramkind: USER (9)
    '''
    #set default extension
    ext = '.txt'
    file_name = os.path.join(folder, feature_type, video_number)

    num_sample = len(features)
    byte_n_sample = num_sample.to_bytes(4, byteorder='big')

    period = 100000
    byte_period = period.to_bytes(4, byteorder = 'big')

    if feature_type == 'COVAREP':
        sample_b = 296
        ext = '.cov'
    elif feature_type == 'WordVec':
        sample_b = 1200
        ext = '.wvec'

    byte_sample_b = sample_b.to_bytes(2, byteorder = 'big')

    sample_type = 9
    byte_sample_type = sample_type.to_bytes(2, byteorder = 'big')
    header = byte_n_sample + byte_period + byte_sample_b + byte_sample_type

    output_byte = b''
    

    for datapoint in features:

        y = list(map(lambda x: struct.pack('>f', x), datapoint))
        byte_datapoint = b''.join(y)

        output_byte += byte_datapoint

    with open(file_name + ext, 'wb') as file:
        file.write(header + output_byte)



In [5]:

import pickle, os, json, codecs


def save_intervals(intervals, feature_type, folder, video_number):
    #set default extension
    ext = '.json'
    video_number = 'intervals_' + video_number
    file_name = os.path.join(folder, feature_type, video_number)
    
    if type(intervals) != list:
        intervals = intervals.tolist()
    json.dump(intervals, codecs.open(file_name + ext, 'w', encoding='utf-8'), indent=4)
    

In [6]:
class new_mmdataset(md.mmdataset):
    #TODO: Need tqdm bar for this as well
    def get_relevant_entries(self,reference):
        relevant_entries={}
        relevant_entries_np={}

        #pbar = tqdm(total=count,unit=" Computational Sequence Entries",leave=False)

        #otherseq_key: OpenFace, wordvec, etc
        for otherseq_key in set(list(self.computational_sequences.keys()))-set([reference]):
            relevant_entries[otherseq_key]={}
            relevant_entries_np[otherseq_key]={}
            sub_compseq=self.computational_sequences[otherseq_key]
            # for some_id in all video ids
            for key in list(sub_compseq.data.keys()):
                keystripped=key.split('[')[0]
                if keystripped not in relevant_entries[otherseq_key]:                           
                    relevant_entries[otherseq_key][keystripped]={}
                    relevant_entries[otherseq_key][keystripped]["intervals"]=[]                     
                    relevant_entries[otherseq_key][keystripped]["features"]=[]                                                            
                    
                relev_intervals=self.computational_sequences[otherseq_key].data[key]["intervals"]                                             
                relev_features=self.computational_sequences[otherseq_key].data[key]["features"]         
                if len(relev_intervals.shape)<2:
                    relev_intervals=relev_intervals[None,:]
                    relev_features=relev_features[None,:]

                relevant_entries[otherseq_key][keystripped]["intervals"].append(relev_intervals)
                relevant_entries[otherseq_key][keystripped]["features"].append(relev_features)
                
            for key in list(relevant_entries[otherseq_key].keys()):
                relev_intervals_np=numpy.concatenate(relevant_entries[otherseq_key][key]["intervals"],axis=0)                                 
                relev_features_np=numpy.concatenate(relevant_entries[otherseq_key][key]["features"],axis=0)
                sorted_indices=sorted(range(relev_intervals_np.shape[0]),key=lambda x: relev_intervals_np[x,0])                               
                relev_intervals_np=relev_intervals_np[sorted_indices,:]                         
                relev_features_np=relev_features_np[sorted_indices,:]

                relevant_entries_np[otherseq_key][key]={}
                relevant_entries_np[otherseq_key][key]["intervals"]=relev_intervals_np
                relevant_entries_np[otherseq_key][key]["features"]=relev_features_np
            log.status("Pre-alignment done for <%s> ..."%otherseq_key)
        return relevant_entries_np
    
    def intersect_and_copy_upsampling(self, ref_all, relevant_entry, epsilon, log_file, feature_info):
        #ref_all: reference_entries[other_key][entry_key]['intervals'] e.g. [COVAREP][some video id]['intervals']
        #relevant_entry: relevant_entries[other_key][entry_key] e.g. [COVAREP][some video id]
        #epsilon: error allowed in alignment
        #ref_time < one interval in relevant_entry
        
        pbar_small=log.progress_bar(total=ref_all.shape[0],unit=" Segments",leave=False)
        pbar_small.set_description("Aligning: " + feature_info)
        
        sub=relevant_entry["intervals"]
        features=relevant_entry["features"]
        
        #finding where intersect happens
        pointer_b = 0 # for relevant_entry
        aligned_sub = []
        aligned_feature = []
        
        for i, inter in enumerate(ref_all):
            print(pointer_b)
            if (abs(inter[0]-inter[1])<epsilon):
                pbar_small.update(1)
                continue
            pointer_c = pointer_b
            while(pointer_c < sub.shape[0]):
                if (inter[0] - sub[pointer_c][0]) > (-epsilon) and (sub[pointer_c][1] - inter[0]) > (-epsilon):
                    aligned_sub.append(sub[pointer_c])
                    aligned_feature.append(features[pointer_c])
                    break
                else:
                    pointer_c += 1
                
            if pointer_c == sub.shape[0]:
                diff = list(map(lambda x: abs(inter[0] - x), sub[:, 0]))
                min_diff = min(diff)
                pointer_c = diff.index(min_diff)
                with open(log_file, 'w+') as fi:
                    fi.write('no corresponding frame, find the closest one, {}, difference: {}\n'.format(feature_info, min_diff))
                aligned_sub.append(sub[pointer_c])
                aligned_feature.append(features[pointer_c])
            else:
                pointer_b = pointer_c

            pbar_small.update(1)
        
        aligned_sub = np.array(aligned_sub)
        aligned_feature = np.array(aligned_feature)
        zero_idx = np.where(np.isinf(aligned_feature))
        aligned_feature[zero_idx] = 0

        pbar_small.close()
        return aligned_sub,aligned_feature
    
    def align_upsampling_and_save(self, reference, id_idx, collapse_function=None, epsilon = 10e-6):
        folder = '/data/mifs_scratch/yw454/cmumosei_aligned'
        log_file = './mosei_alignment_log.txt'
        #aligned_output = {}
        count = 0
        
        ##self.computational_sequences.keys are COVERAP, OpenFace, WordVec, etc
        #for sequence_name in self.computational_sequences.keys():
        #    #init a dictionary to store different featues seperately
        #    aligned_output[sequence_name]={}
        
        if reference not in self.computational_sequences.keys():
            log.error("Computational sequence <%s> does not exist in dataset"%reference,error=True)
        
        #get data of reference feature
        refseq=self.computational_sequences[reference].data
        #unifying the dataset, removing any entries that are not in the reference computational sequence
        self.unify()
        
        #building the relevant entries to the reference - what we do in this section is simply removing all the [] from the entry ids and populating them into a new dictionary
        log.status("Pre-alignment based on <%s> computational sequence started ..."%reference)
        
        relevant_entries=self.get_relevant_entries(reference)
        log.status("Alignment starting ...")
        
        
        pbar = log.progress_bar(total=len(refseq.keys()),unit=" Computational Sequence Entries",leave=False)
        pbar.set_description("Overall Progress")
        # for some_id in all video ids
        for entry_key in list(refseq.keys()):
            
            if entry_key in id_idx:
                stored_idx = id_idx.index(entry_key)
                if stored_idx < 104 or (stored_idx > 104 and stored_idx < 1781):
                    continue
            
            all_intersects = {}
            all_intersect_features = {}
            
            for sequence_name in self.computational_sequences.keys():
                all_intersects[sequence_name] = []
                all_intersect_features[sequence_name] = []
            
            ref_all=refseq[entry_key]['intervals']
                
            #aligning all sequences to ref sequence (previous: align refer to refer as well, now: not include refer)
            #otherseq_key: other features; entry_key: some video id
                
            for otherseq_key in list(self.computational_sequences.keys()):
                if otherseq_key != reference:
                    feature_info = 'reference: {}, other feature {}, video id: {}'.format(reference, otherseq_key, entry_key)
                    intersects,intersects_features=self.intersect_and_copy_upsampling(ref_all,relevant_entries[otherseq_key][entry_key],epsilon, log_file, feature_info)
                else:
                    intersects,intersects_features=refseq[entry_key]['intervals'],refseq[entry_key]['features']                    
                    
                #print(type(intersects[0]))
                #print(type(intersects_features[0]))
                #print(len(intersects[0]))
                #print(len(intersects_features[0]))
                all_intersects[otherseq_key].append(intersects)
                all_intersect_features[otherseq_key].append(intersects_features)
                    

            #save features per video
            for sequence_name in self.computational_sequences.keys():
                video_code = id_idx.index(entry_key)
                video_code = str(video_code).zfill(6)
                
                save_htk_format(all_intersect_features[sequence_name], sequence_name, folder, video_code)
                save_intervals(all_intersects[sequence_name], sequence_name, folder, video_code)
                print('alignment saved for video {}.'.format(video_code))
            
            pbar.update(1)
        pbar.close()




In [7]:
# define your different modalities - refer to the filenames of the CSD files
basic_dict={'COVAREP': DATA_PATH + 'CMU_MOSEI_COVAREP.csd', 
            'WordVec': DATA_PATH + 'CMU_MOSEI_TimestampedWordVectors.csd'}
second_dict = {'Facet': DATA_PATH + 'CMU_MOSEI_VisualFacet42.csd',
            'OpenFace': DATA_PATH + 'CMU_MOSEI_VisualOpenFace2.csd'}
other_dict = {'Word': DATA_PATH + 'CMU_MOSEI_TimestampedWords.csd',
             'Phone': DATA_PATH + 'CMU_MOSEI_TimestampedPhones.csd'}
label_dict = {'mylabels':DATA_PATH + 'CMU_MOSEI_Labels.csd'}

In [8]:
# list the directory contents... let's see what features there are
data_files = os.listdir(DATA_PATH)
print('\n'.join(data_files))

CMU_MOSEI_VisualOpenFace2.csd
CMU_MOSEI_TimestampedWords.csd
CMU_MOSEI_COVAREP.csd
CMU_MOSEI_VisualFacet42.csd
CMU_MOSEI_TimestampedWordVectors.csd
CMU_MOSEI_TimestampedPhones.csd
CMU_MOSEI_Labels.csd
COVAREP.zip


In [24]:
basic_dataset = new_mmdataset(basic_dict)
second_dataset = new_mmdataset(second_dict)
label_dataset = new_mmdataset(label_dict)

validators eliminated
[2021-01-23 22:47:44.619] | Success | Computational sequence read from file /data/mifs_scratch/yw454/cmu_mosei_latest/CMU_MOSEI_COVAREP.csd ...
[2021-01-23 22:47:44.818] | Warning | Validation of the computational sequence skipped by user request.
validators eliminated
[2021-01-23 22:47:44.819] | Success | Computational sequence read from file /data/mifs_scratch/yw454/cmu_mosei_latest/CMU_MOSEI_TimestampedWordVectors.csd ...
[2021-01-23 22:47:44.963] | Warning | Validation of the computational sequence skipped by user request.
[2021-01-23 22:47:44.963] | Success | Dataset initialized successfully ... 
validators eliminated
[2021-01-23 22:47:44.964] | Success | Computational sequence read from file /data/mifs_scratch/yw454/cmu_mosei_latest/CMU_MOSEI_VisualFacet42.csd ...
[2021-01-23 22:47:45.105] | Warning | Validation of the computational sequence skipped by user request.
validators eliminated
[2021-01-23 22:47:45.106] | Success | Computational sequence read from 

In [25]:
[COVAREP, WordVec] = [basic_dataset.computational_sequences['COVAREP'],
                        basic_dataset.computational_sequences['WordVec']]
label = label_dataset.computational_sequences['mylabels']

In [38]:
not_enough_word = set()
not_enough_label = set()
no_labels = set()
label_keys = list(label.keys())
for k in list(COVAREP.keys()):
    acoustic = COVAREP[k]['intervals'][-1][1]
    language = WordVec[k]['intervals'][-1][1]
    
    if language / acoustic > 2 or acoustic / language > 2:
        not_enough_word.add(k)
    
    if k in label_keys:
        annotations = label[k]['intervals'][-1][1] - label[k]['intervals'][0][0]
    else:
        no_labels.add(k)
    
    if annotations / acoustic > 2 or acoustic / annotations > 2:
        not_enough_label.add(k)
    
print(len(not_enough_label))
print(len(not_enough_word))
print(len((not_enough_label & not_enough_word)))
print(len(no_labels))   
        
    


/home/dawna/ql264/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars


1403
295
293
544


In [32]:
print(label['-3nNcZdcdvU']['intervals'][0])


[0.    9.213]


KeyError: 0

In [13]:
video_id_record = os.path.join('/data/mifs_scratch/yw454/cmumosei_aligned', 'video_id.json')

with open(video_id_record, 'r') as json_file:
    video_ids = json.load(json_file)

basic_dataset.align_upsampling_and_save('COVAREP', video_ids)



[2021-01-23 21:03:21.656] | Status  | Unify was called ...
[2021-01-23 21:03:21.665] | Warning | PEBwwe0PLZ8 entry is not shared among all sequences, removing it ...
[2021-01-23 21:03:21.668] | Success | Unify completed ...
[2021-01-23 21:03:21.668] | Status  | Pre-alignment based on <COVAREP> computational sequence started ...


  0%|          | 0/59926 [00:00<?, ? Segments/s]
Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0:   0%|          | 0/59926 [00:00<?, ? Segments/s]
Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0:   0%|          | 168/59926 [00:00<00:35, 1679.74 Segments/s]

[2021-01-23 21:03:42.726] | Status  | Pre-alignment done for <WordVec> ...
[2021-01-23 21:03:43.586] | Status  | Alignment starting ...
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
1


Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0:   1%|          | 547/59926 [00:00<00:29, 2015.93 Segments/s]
Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0:   1%|▏         | 864/59926 [00:00<00:27, 2139.01 Segments/s]


17
17
17
17
17
17
18
18
18
18
18
18
18
18
19
19
19
19
19
19
19
19
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
21
21
21
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
25
25
25
25
25
25
25
25
25
25
25
25
25
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
28
28
28
28
28
28
28
28
28
28
28
28
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
29
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
31
3

...


Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0: 100%|█████████▉| 59912/59926 [19:45<00:00, 28.76 Segments/s]
Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0: 100%|█████████▉| 59917/59926 [19:45<00:00, 31.83 Segments/s]

193
193
193
193
193
193
193
193
193



Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0: 100%|█████████▉| 59921/59926 [19:46<00:00, 29.63 Segments/s]
Aligning: reference: COVAREP, other feature WordVec, video id: 0y022OlZ3W0: 100%|██████████| 59926/59926 [19:46<00:00, 32.55 Segments/s]

193
193
193
193
193


error: required argument is not a float

In [44]:
some_id = list(WordVec.keys())[10]
print(WordVec[some_id]['features'][0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
